In [ ]:
import requests
import pandas as pd

In [ ]:
df1 = pd.read_csv('train_data_pajak.csv')
df2 = pd.read_csv('clean_data.csv')
def run_curl_command(prompt):
    url = "http://108.137.109.168:11434/api/generate"

    data = {
        "model": "mistral",
        "prompt": prompt,
        "stream": False
    }

    # Sending a POST request
    response = requests.post(url, json=data)

    # Checking the response
    if response.status_code == 200:
        # Return the JSON response
        return response.json()
    else:
        print(f"Error! Status code: {response.status_code}, Response: {response.text}")
        # Return None or raise an exception, depending on your preference
        return None

In [30]:
df2.head()

,NAMA_BARANG,NAMA_BARANG_NO_SPCHAR,JML_BARANG,KRITERIA_BARANG
0,ANALISA SAMPEL RBD PALM STEARIN,ANALISA SAMPEL RBD PALM STEARIN,1.0,NaN
1,Aflatoxin B1 Qty 1 Sampel Palm Kernel Expell...,AFLATOXIN B1 QTY 1 SAMPEL PALM KERNEL EXPELL...,1.0,NaN
2,Analisa Pengujian INSPEKSI MINYAK KELAPA SAWI...,ANALISA PENGUJIAN INSPEKSI MINYAK KELAPA SAWI...,1.0,NaN
3,Discarging Survey of CPO 2999998 MT at Kabil B...,DISCARGING SURVEY OF CPO 2 999 998 MT AT KABIL...,1.0,CPO
4,Discarging Survey of CPO 3149193 MT at Kabil B...,DISCARGING SURVEY OF CPO 3 149 193 MT AT KABIL...,1.0,CPO


In [31]:
delete_column_df2 = ['NAMA_BARANG_NO_SPCHAR', 'KRITERIA_BARANG']
df2.drop(delete_column_df2, axis=1, inplace=True)

In [33]:
df2.to_csv('clean_dataset_pajak.csv', index=False)

In [35]:
df3= pd.read_csv ('clean_dataset_pajak.csv')

df3.head()

,NAMA_BARANG,JML_BARANG
0,ANALISA SAMPEL RBD PALM STEARIN,1.0
1,Aflatoxin B1 Qty 1 Sampel Palm Kernel Expell...,1.0
2,Analisa Pengujian INSPEKSI MINYAK KELAPA SAWI...,1.0
3,Discarging Survey of CPO 2999998 MT at Kabil B...,1.0
4,Discarging Survey of CPO 3149193 MT at Kabil B...,1.0


In [36]:
new_header = ['NamaBarang', 'JumlahBarang']
df3.columns = new_header

In [38]:
df3.to_csv('clean_dataset_pajak_header.csv', index=False)

In [39]:
df4 = pd.read_csv ('clean_dataset_pajak_header.csv')

In [ ]:
result_list=[]
for i in range (0,50,10):
    result = run_curl_command("""
Please make classification for the column BARANG in this data {y} based on the following criteria:
- type represents the type of classification ('Service' or 'Product').
- If the product contains a verb, an action or assisstance, classify it as 'service'.
- If the product is raw material or processed from palm oil, classify it as 'product'.
- Consider {x} as the train data.
- ProductName represents the NamaBarang column in {y}.
- AmountOfProduct represents the JumlahBarang column in {y}.
- ConfidenceLevel represents the confidence level of the classification.
- explanation provides additional information about the classification.
- please remember that we only do all of those prompt above based on the data under the "NAMA_BARANG"
- Only return the JSON formatted answer and nothing else.
example JSON format:
    {{
        "productName":"Analisa  Pengujian INSPEKSI MINYAK KELAPA SAWIT DAN PRODUK TURUNANNYA dan Analisa  Pengujian 1 Sa",
        "type":"Service",
        "confidenceLevel":0.9,
        "explanation":"This is....",
        ...
    }},
    {{
        "productName":"TANDAN BUAH SEGAR KELAPA SAWIT TAHUN TANAM 1999 1700094 KG",
        "type":"Product",
        "confidenceLevel":0.9,
        "explanation":"This is a product of Fresh Fruit Oil Palm....",
        ...
    }},
    {{
        "productName": "BLANKO SOUNDING PALM KERNEL",
        "type": "Service",
        "confidenceLevel": 0.9,
        "explanation": "This is a sounding service for palm kernel."
        ...
    }}
    """.format(y=df4[i:i+10], x=df1))['response']
    print(result)
    result_list.append(result)

In [67]:
df_results = pd.DataFrame(result_list)
print(df_results)

                                                   0
0   [\n{\n"productName": "ANALISA SAMPEL RBD PALM...
1   [\n{\n"productName": "Pengiriman Kecambah Saw...
2   [\n{\n"productName": "TANDAN BUAH SEGAR KELAP...
3   [\n{\n"productName": "TANDAN BUAH SEGAR KELAP...
4   [\n{\n"productName": "Minyak Kelapa Sawit CPO...


In [70]:
    for result in result_list:
        print(result)  # Debugging print
    df_results = pd.concat([pd.DataFrame(result) for result in result_list], ignore_index=True)

 [
{
"productName": "ANALISA SAMPEL RBD PALM STEARIN",
"type": "Service",
"confidenceLevel": 1.0,
"explanation": "This is a service of RBD Palm Stearin Analysis."
},
{
"productName": "Aflatoxin B1 Qty 1 Sampel Palm Kernel Expell",
"type": "Service",
"confidenceLevel": 1.0,
"explanation": "This is a service of Aflatoxin B1 Quantity inspection for Palm Kernel Expelled."
},
{
"productName": "Analisa Pengujian INSPEKSI MINYAK KELAPA SAWI",
"type": "Service",
"confidenceLevel": 1.0,
"explanation": "This is a service of Palm Oil Sawinseed Inspection and Testing."
},
{
"productName": "Discarging Survey of CPO 2999998 MT at Kabil B",
"type": "Service",
"confidenceLevel": 1.0,
"explanation": "This is a service of CPO Discharging Survey at Kabil B."
},
{
"productName": "Discarging Survey of CPO 3149193 MT at Kabil B",
"type": "Service",
"confidenceLevel": 1.0,
"explanation": "This is a service of CPO Discharging Survey at Kabil B."
},
{
"productName": "Kalibrasi Sounding Tape CPO",
"type": "Serv

ValueError: DataFrame constructor not properly called!

In [71]:
df_results.to_csv('results.csv', index=False)

In [72]:
print(df_results)

                                                   0
0   [\n{\n"productName": "ANALISA SAMPEL RBD PALM...
1   [\n{\n"productName": "Pengiriman Kecambah Saw...
2   [\n{\n"productName": "TANDAN BUAH SEGAR KELAP...
3   [\n{\n"productName": "TANDAN BUAH SEGAR KELAP...
4   [\n{\n"productName": "Minyak Kelapa Sawit CPO...


In [ ]:
for i in range (0,len(df2)-100,100):
    result = run_curl_command("""
    Please generate a JSON format only (ready to convert from string to JSON) of classification for the column NAMA_BARANG in this data {y} based on the following criteria:
- If the product contains a verb, classify it as 'service'.
- If the product is raw material or processed from palm oil, classify it as 'product'.
- Consider {x} as the train data.
- product_name represents the NAMA_BARANG column in {y}.
- amount_of_product represents the JML_BARANG column in {y}.
- type represents the type of classification ('Service' or 'Product').
- date, location, year_planted, weight, and Unit_of_Measurement are optional attributes depens on the information that could be retrieve from the data.
- confidence_level represents the confidence level of the classification.
- explanation provides additional information about the classification. 
example format:
    [{{
        'product_name':'Analisa  Pengujian INSPEKSI MINYAK KELAPA SAWIT DAN PRODUK TURUNANNYA dan Analisa  Pengujian 1 Sa',
        'amount_of_product' : 0,
        'type':'Service',
        'date' : NaN,
        'location' : NaN,
        'year_planted': NaN,
        'weight': ,
        'Unit_of_Measurement' : NaN,
        'Confidence_level':0.9,
        'explanation':'This is....',
    }},{{
        'product_name':'TANDAN BUAH SEGAR KELAPA SAWIT TAHUN TANAM 1999 1700094 KG',
        'amount_of_product' : 1,
        'type':'Product',
        'date' : NaN,
        'location' : NaN,
        'year_planted': 1999,
        'weight': 1700094,
        'Unit_of_Measurement' : KG,
        'confidence_level':0.9,
        'explanation':'This is a product of Fresh Fruit Oil Palm....',
    }}]

    """.format(y=df2[i:i+100], x=df1))['response']
    print(result)

In [ ]:

for i in range (0,10,2):
    result = run_curl_command("""
    Please generate a JSON format of classification for the column NAMA_BARANG in this data {y} based on the following criteria:
- If the product contains a verb, classify it as 'service'.
- If the product is raw material or processed from palm oil, classify it as 'product'.
- Consider {x} as the train data.
- product_name represents the NAMA_BARANG column in {y}.
- amount_of_product represents the JML_BARANG column in {y}.
- type represents the type of classification ('Service' or 'Product').
- date, location, year_planted, weight, and Unit_of_Measurement are optional attributes depens on the information that could be retrieve from the data.
- confidence_level represents the confidence level of the classification.
- explanation provides additional information about the classification. 
example format:
    [{{
        'product_name':'Analisa  Pengujian INSPEKSI MINYAK KELAPA SAWIT DAN PRODUK TURUNANNYA dan Analisa  Pengujian 1 Sa',
        'amount_of_product' : 0,
        'type':'Service',
        'date' : NaN,
        'location' : NaN,
        'year_planted': NaN,
        'weight': ,
        'Unit_of_Measurement' : NaN,
        'Confidence_level':0.9,
        'explanation':'This is....',
        ....
    }},{{
        'product_name':'TANDAN BUAH SEGAR KELAPA SAWIT TAHUN TANAM 1999 1700094 KG',
        'amount_of_product' : 1,
        'type':'Product',
        'date' : NaN,
        'location' : NaN,
        'year_planted': 1999,
        'weight': 1700094,
        'Unit_of_Measurement' : KG,
        'confidence_level':0.9,
        'explanation':'This is a product of Fresh Fruit Oil Palm....',
        ....
    }}]""".format(y=df2[i:i+2], x=df1))['response']
    print(result)

In [ ]:
print(result)

In [ ]:
prompt=""" based on the column of NAMA_BARANG and JML_BARANG following data: {x}, 
    generate a json format for each data and breakdown the text such as:
    [{{
        'JML_BARANG' : xx,
        'UOM' : aa,
        'Date' : bb,
        'location' : ccc,
        'year_planted': dd,
        ....
    }},
    {{
        'JML_BARANG' : xx,
        'UOM' : aa,
        'Date' : bb,
        'location' : ccc,
        'year_planted': dd,
        ....
    }}]""".format(x=result)
print(run_curl_command(prompt))
print("DONE")

In [ ]:
run_curl_command(response1)

In [ ]:
response2=prompt
prompt = """
please breakdown text of column NAMA_BARANG and JML_BARANG in {y} into this format:

"""